In [36]:
from IPython.display import Image


In [29]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
assert load_dotenv()

In [3]:
Image(url='https://wangwei1237.github.io/2023/09/20/Introduction-to-LangChain/langchain.png', width=500)

- https://github.com/hwchase17/langchain-0.1-guides/blob/master/output_parsers.ipynb
- lcel => agent
  - variable assignment
  - prompt template
  - llm (with tools)
  - output parse

In [4]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [5]:
class Ingredient(BaseModel):
    name: str = Field(description="The name of the ingredient")
    quantity: str = Field(description="The specific unit of measurement corresponding to the quantity, such as grams, ounces, liters, etc.")
    unit: str = Field(description="The amount of the ingredient required for the recipe. This can be represented using various units such as grams, cups, teaspoons, etc.")

class Recipe(BaseModel):
    name: str = Field(description="The name of the recipe")
    ingredients: List[Ingredient] = Field(description="The list of ingredients for the recipe")

In [6]:
parser = PydanticOutputParser(pydantic_object=Recipe)

In [7]:
from rich.pretty import pprint
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Ingredient": {"properties": {"name": {"description": "The name of the ingredient", "title": "Name", "type": "string"}, "quantity": {"description": "The specific unit of measurement corresponding to the quantity, such as grams, ounces, liters, etc.", "title": "Quantity", "type": "string"}, "unit": {"description": "The amount of the ingredient required for the recipe. This can be represented using various units such as grams, cups, teaspoons, etc.", "title": "Unit", "type": "string"}}, "required": ["name", "quantity", "unit"], "title

### converting messages

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI(model='gpt-3.5-turbo')

In [10]:
chain = prompt | model

In [13]:
chain.invoke({'topic': 'pig'})

AIMessage(content='Why did the pig go to the casino?\n\nTo play the slop machine!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C7OolDfPxp7vrrgFm16X88ZS9Odnv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--575bb30c-6c05-4758-8ce5-c121564597f5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
from langchain_core.output_parsers import StrOutputParser

In [15]:
parser = StrOutputParser()
chain |= parser

In [16]:
chain.invoke({'topic': 'pig'})

'Why did the pig go to the casino? Because he heard they had a lot of bacon!'

In [17]:
chain

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000017F87BD0A90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000017F87BD0EE0>, root_client=<openai.OpenAI object at 0x0000017F87BD05E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000017F87BD0460>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [18]:
chain = prompt | model | parser
chain.invoke({'topic': 'pig'})

'Why did the pig go to the casino? To bring home the bacon!'

In [19]:
chain = {'topic': lambda x:x['input']} | prompt | model | parser
chain.invoke({'input': 'apple'})

'Why did the apple stop in the middle of the road? \n\nBecause it ran out of juice!'

### OpenAI Function Calling

In [20]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, validator

In [21]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

openai_functions = [convert_to_openai_function(Joke)]
openai_functions

[{'name': 'Joke',
  'description': 'Joke to tell user.',
  'parameters': {'properties': {'setup': {'description': 'question to set up a joke',
     'type': 'string'},
    'punchline': {'description': 'answer to resolve the joke',
     'type': 'string'}},
   'required': ['setup', 'punchline'],
   'type': 'object'}}]

In [22]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [23]:
parser = JsonOutputFunctionsParser()

In [24]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [25]:
chain.invoke({'topic': 'pig'})

{'setup': 'What do you call a pig that knows karate?',
 'punchline': 'A pork chop!'}

### PydanticOutputParser

In [27]:
class WritingScore(BaseModel):
    readability: int
    conciseness:int

In [32]:
schema = WritingScore.model_json_schema()
schema

{'properties': {'readability': {'title': 'Readability', 'type': 'integer'},
  'conciseness': {'title': 'Conciseness', 'type': 'integer'}},
 'required': ['readability', 'conciseness'],
 'title': 'WritingScore',
 'type': 'object'}

In [33]:
resp = """```
{
  "readability": 8,
  "conciseness": 9
}
```"""

In [34]:
parser = PydanticOutputParser(pydantic_object=WritingScore)

In [35]:
parser.parse(resp)

WritingScore(readability=8, conciseness=9)